In [ ]:
from tiger.io import read_image, write_image
from tiger.resampling import pad_or_crop_image, resample_mask_itk, change_direction
import numpy as np
import os
import SimpleITK as sitk
import matplotlib.pyplot as plt

## Using vertebra-segmentation:verse2019processor 

In [ ]:
img_dir_mha = '/mnt/netcache/bodyct/experiments/vertebra_fracture_detection_t9560/datasets/verse2019/input_extra_masks/mha/'
img_dir_nifti = '/mnt/netcache/bodyct/experiments/vertebra_fracture_detection_t9560/datasets/verse2019/input_extra_masks/nifti/'
img_paths_mha = [os.path.join(img_dir_mha, file) for file in sorted(os.listdir(img_dir_mha))]

msk_dir_mha = '/mnt/netcache/bodyct/experiments/vertebra_fracture_detection_t9560/datasets/verse2019/verse2019processor_masks/mha/'
msk_dir_nifti = '/mnt/netcache/bodyct/experiments/vertebra_fracture_detection_t9560/datasets/verse2019/verse2019processor_masks/nifti/'
msks_paths_nifti = [os.path.join(msk_dir_nifti, file) for file in sorted(os.listdir(msk_dir_nifti)) if 'nii' in file]
msks_paths_mha = [os.path.join(msk_dir_mha, file) for file in sorted(os.listdir(msk_dir_mha))]

In [ ]:
# read all mha files and transform to nifti
# for img_path_mha in img_paths_mha:
#     verse_id = img_path_mha.split('/')[-1].split('.')[0]
#     img, header_img = read_image(img_path_mha)
#     out_file = os.path.join(img_dir_nifti, verse_id) + '.nii.gz'
#     print(out_file)
#     write_image(out_file, img, header_img)

In [ ]:
# read the nifti files and convert back to mha
for mask_path in msks_paths_nifti:
    verse_id = mask_path.split('/')[-1].split('.')[0].split('_')[0]
    msk, header_msk = read_image(mask_path)
    out_file = os.path.join(msk_dir_mha, verse_id) + '.mha'
    print(out_file)
    write_image(out_file, msk, header_msk)

## Lets check the masks!

In [ ]:
for i, (img_path, msk_path) in enumerate(zip(img_paths_mha, msks_paths_mha)):
    # read the image and the predicted mask
    img, header_img = read_image(img_path)
    msk, header_msk = read_image(msk_path)
    file_name = img_path.split('/')[-1]
    
    # filter out non C vertebrae, otherwise the vertebrae get the same color
    msk = np.where(msk < 26, msk, 0)
    plt_msk = np.ma.masked_where(msk == 0, msk)
    
    # mid slice of the image
    mid_slice = img.shape[0] // 2 

    plt.figure(figsize=(15, 30))
    plt.imshow(img[mid_slice, :, :], cmap='gray')
    plt.imshow(plt_msk[mid_slice, :, :], alpha=0.25)
    plt.title(file_name)
    plt.show()

## Corrections need to be applied:
Corrections:
* Case: 2
  'The vertebra with label T2 is actually T1'
* Case 7
  'This is the case with an L6 vertebra. L5 is actually L6, T12 is actually L1'
* Case: 14 
  'Segmentation error, person probably has 6 lumbar vertebrae but L6 and L5 are both labeled as "L4" (brown)'
* Case: 29
  'C1 is not segmented and C2 is labeled as C1, etc'
* Case: 31
  'Segmentation of C1 is missing, labeling is overall correct but T3 and T4 are both labeled as T3'
* Case: 33
  'C1 is missing, T3 and T4 are both labeled as T3'
* Case: 35
  'The vertebra labeled as T2 is actually T1. This person seems to have only 4 lumbar vertebrae.'

In [ ]:
msk_path = [f for f in msks_paths_mha if '011' in f][0]

msk, header_msk = read_image(msk_path)
out_file = os.path.join(msk_dir_mha, 'corrected_' + msk_path.split('/')[-1])
print(out_file)
msk = np.where(msk > 0, msk - 1, 0)
write_image(out_file, msk, header_msk)